# Diseño de experimento

El diseño de experimentos en el entrenamiento de redes neuronales juega un papel crucial en la optimización del rendimiento del modelo. Este enfoque sistemático permite explorar y ajustar los hiperparámetros (como la tasa de aprendizaje, el número de capas y neuronas, y el tamaño del lote) de manera eficiente y efectiva. Al planificar y estructurar los experimentos, se pueden identificar las combinaciones óptimas de hiperparámetros que mejoran la precisión y la generalización del modelo, mientras se minimiza el tiempo y los recursos computacionales necesarios. Además, un buen diseño de experimentos reduce el impacto ecológico del entrenamiento de redes neuronales al disminuir el número de ejecuciones necesarias, optimizando así el consumo de energía y hardware.

## Una variable a la vez

El enfoque de diseño de experimentos de una variable a la vez (One Variable At a Time, OVAT) es una metodología simple en la que se varía un solo factor o variable experimental mientras se mantienen constantes todos los demás factores. Este enfoque permite observar cómo cambios en esa única variable afectan el resultado del experimento. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/irvingvasquez/practicas_pytorch/blob/master/06_doe_una_variable.ipynb)

Si ejecutas en COLAB debes copiar los archivos extra de este repositorio.

@juan1rving


In [1]:
# Cargamos paquetes necesarios

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np
import time

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms

#Types
from torch.utils.data import DataLoader, Dataset
from typing import Dict, Tuple, Union
from torch import optim, nn

#helper was developed by Udacity under MIT license
import helper

### 1. Definición de variables y rango de valores:

Identificar los factores o variables experimentales clave que se desean estudiar. En el contexto de redes neuronales, estas variables pueden incluir la tasa de aprendizaje, el número de capas, el número de neuronas por capa, el tamaño del lote, entre otros.

Variables que tomaremos en cuenta:

- Tasa de aprendizaje (eta)
- Número de épocas (n_epocas)
- Tamaño de lote (batch_size)

Una vez definidas las variables independientes definimos un rango de valores posibles para cada variable.

> TODO: Define un rango de valores posibles para cada variable. Incluye el valor mínimo y el valor máximo. Se sugiere utilizar una lista de valores obtenida con una separación uniforme. Probar almenos 5 valores por variable.




In [2]:
hyperparameter_range = {
    'eta' : [1e-4, 3e-4, 1e-3, 3e-3, 1e-2],
    'epochs' : [1, 2, 3, 4, 5],
    'batch_size' : [16, 32, 64, 128, 256]
}

### 2. Configuración inicial:

Establecer una configuración inicial para la red neuronal con valores predeterminados para todos los hiperparámetros. 

> TODO: Define la configuración inicial. Se sugiere usar un diccionario para contener dicha configuración.


In [3]:
configuration = {
    'eta': 0.001,
    'epochs': 1,
    'batch_size': 32
}

best_configuration = {
    'eta': None,
    'epochs': None,
    'batch_size': None
}

best_accuracy = {
    'eta': float('-inf'),
    'epochs': float('-inf'),
    'batch_size': float('-inf')
}


### 3. Variación de una variable a la vez:

- Seleccionar la primera variable a estudiar (por ejemplo, la tasa de aprendizaje).
- Realizar una serie de experimentos donde se varía únicamente la tasa de aprendizaje, mientras se mantienen constantes todos los demás hiperparámetros.
- Registrar el rendimiento del modelo para cada valor de la tasa de aprendizaje.

> TODO: Modifica el código para que pueda aceptar la configuración deseada

In [4]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5), (0.5))])
trainset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=True, transform=transform)
validationset = datasets.FashionMNIST('F_MNIST_data/', download=True, train=False, transform=transform)

In [5]:
class RedNeuronal(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p = 0.5):
        '''
        Construye una red de tamaño arbitrario.
        
        Parámetros:
        input_size: cantidad de elementos en la entrada
        output_size: cantidada de elementos en la salida 
        hidden_layers: cantidad de elementos por cada capa oculta
        drop_p: probabilidad de "tirar" (drop) una neurona [0,1] 
        '''
        # llamamos al constructor de la superclase
        super().__init__()
        
        # Agregamos la primera capa
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        # agregamos cada una de las capas, zip empareja el número de entradas con las salidas
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        # agregamos la capa de salida final de la red
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        # Incluimos drop-out en la red
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        ''' Pase hacia adelante en la red, el regreso son las probabilidades en el dominio log '''
        
        # Hacemos un pase frontal en cada una de las capas ocultas, 
        # La funció de activación es un RELU combinado con dropout
        for linear in self.hidden_layers:
            x = F.relu(linear(x))
            x = self.dropout(x)
        
        x = self.output(x)
        
        return F.log_softmax(x, dim=1)

In [6]:
# Create the network, define the criterion and optimizer
model = RedNeuronal(784, 10, [516, 256], drop_p=0.5)
criterion = nn.NLLLoss()

In [7]:
def get_hyperparameters(
    configuration: Dict[str, Union[float, int]],
    model: nn.Module,
    trainset: Dataset,
    validationset: Dataset
) -> Tuple[optim.Optimizer, int, DataLoader, DataLoader]:

    batch_size = configuration["batch_size"]
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
    validationloader = DataLoader(validationset, batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(model.parameters(), lr=configuration["eta"])

    return optimizer, configuration["epochs"], trainloader, validationloader


In [8]:
# Implementamos una función de evaluación
def validation(model, validationloader, criterion):
    test_loss = 0
    accuracy = 0
    for images, labels in validationloader:

        images.resize_(images.shape[0], 784)

        output = model.forward(images)
        test_loss += criterion(output, labels).item()

        ps = torch.exp(output)
        equality = (labels.data == ps.max(dim=1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [ ]:
def train_model(model: nn.Module, optimizer : optim.Optimizer, epochs : int, trainloader : DataLoader, validationloader :  DataLoader, criterion):
    steps = 0
    running_loss = 0
    print_every = 40
    for e in range(epochs):
    # Cambiamos a modo entrenamiento
        model.train()
        for images, labels in trainloader:
            steps += 1
    
            # Aplanar imágenes a un vector de 784 elementos
            images.resize_(images.size()[0], 784)
            
            optimizer.zero_grad()
            
            output = model.forward(images)
            loss = criterion(output, labels)
            # Backprogamation
            loss.backward()
            # Optimización
            optimizer.step()
            
            running_loss += loss.item()
            
            if steps % print_every == 0:
                # Cambiamos a modo de evaluación
                model.eval()
                
                # Apagamos los gradientes, reduce memoria y cálculos
                with torch.no_grad():
                    test_loss, accuracy = validation(model, validationloader, criterion)
                    
                print("Epoch: {}/{}.. ".format(e+1, epochs),
                    "Pérdida de entrenamiento: {:.3f}.. ".format(running_loss/print_every),
                    "Pérdida de validación: {:.3f}.. ".format(test_loss/len(validationloader)),
                    "Exactitud de validación: {:.3f}".format(accuracy/len(validationloader)))
                
                running_loss = 0
                
                # Make sure training is back on
                model.train()


In [ ]:
def reset_weights(m):
    if hasattr(m, "reset_parameters"):
        m.reset_parameters()

In [10]:
experiments = []
id_of_experiment = 0

for hyper_name, values in hyperparameter_range.items():

    print(f"\n>>> Probando hiperparámetro: {hyper_name}")
    print(f"Valores a probar: {values}\n")

    for value in values:
        print(f"  - Probando {hyper_name} = {value}")

        configuration[hyper_name] = value
        model.apply(reset_weights)
        
        optimizer, epochs, trainloader, validationloader = get_hyperparameters(
            configuration=configuration,
            model=model,
            trainset=trainset,
            validationset=validationset
        )

        train_model(
            optimizer = optimizer, 
            epochs = epochs, 
            trainloader = trainloader, 
            validationloader = validationloader,
            criterion = criterion
        )
        
        test_loss, test_accuracy = validation(
            model=model,
            validationloader=validationloader,
            criterion=criterion
        )

        print(f"    -> Accuracy obtenida: {test_accuracy:.4f}")

        experiments.append({
            "id_of_experiment": id_of_experiment,
            "eta": configuration["eta"],
            "epochs": configuration["epochs"],
            "batch_size": configuration["batch_size"],
            "accuracy": test_accuracy
        })

        if test_accuracy > best_accuracy[hyper_name]:
            best_accuracy[hyper_name] = test_accuracy
            best_configuration[hyper_name] = value
            print(f"    * Nuevo mejor valor encontrado para {hyper_name}: {value}")

        id_of_experiment += 1

    print(f"\n>>> Mejor valor para {hyper_name}: {best_configuration[hyper_name]}")
    configuration[hyper_name] = best_configuration[hyper_name]


Epoch: 1/1..  Pérdida de entrenamiento: 2.117..  Pérdida de validación: 1.858..  Exactitud de validación: 0.578
Epoch: 1/1..  Pérdida de entrenamiento: 1.663..  Pérdida de validación: 1.324..  Exactitud de validación: 0.645
Epoch: 1/1..  Pérdida de entrenamiento: 1.270..  Pérdida de validación: 1.034..  Exactitud de validación: 0.658
Epoch: 1/1..  Pérdida de entrenamiento: 1.109..  Pérdida de validación: 0.892..  Exactitud de validación: 0.710
Epoch: 1/1..  Pérdida de entrenamiento: 0.972..  Pérdida de validación: 0.815..  Exactitud de validación: 0.727
Epoch: 1/1..  Pérdida de entrenamiento: 0.920..  Pérdida de validación: 0.768..  Exactitud de validación: 0.733
Epoch: 1/1..  Pérdida de entrenamiento: 0.880..  Pérdida de validación: 0.732..  Exactitud de validación: 0.737
Epoch: 1/1..  Pérdida de entrenamiento: 0.858..  Pérdida de validación: 0.716..  Exactitud de validación: 0.741
Epoch: 1/1..  Pérdida de entrenamiento: 0.851..  Pérdida de validación: 0.690..  Exactitud de validación

KeyboardInterrupt: 

### 4. Selección del mejor valor:

Analizar los resultados y seleccionar el valor de la tasa de aprendizaje que produce el mejor rendimiento del modelo.


### 5. Repetición para otras variables:

Proceder con la siguiente variable (por ejemplo, el número de capas) y repetir el proceso: variar solo esta variable mientras se mantienen constantes todos los demás hiperparámetros, utilizando el mejor valor encontrado para la tasa de aprendizaje. Continuar este proceso para cada variable en la lista.

> TODO: Escribe una tabla con el resultado de cada experimento. Las columnas deben ser: ID, Configuración, Exactitud obtenida.

In [ ]:
import pandas as pd
from IPython.display import display

df_experiments = pd.DataFrame(experiments)
display(df_experiments)


## Conclusiones

¿Cual fue el mejor valor encontrado?
¿Cuantas ejecuciones se realizaron?
¿Que tiempo tomó realizar todos los experimentos?